In [17]:
import numpy as np
from scipy.sparse.linalg import lgmres
from numpy import linalg as la
from scipy.sparse.linalg import LinearOperator
import matplotlib.pyplot as plt

In [18]:
def Arnoldi(A, v, m):
    w=[]
    V = []
    V.append(np.array(v[:]))
    h=np.zeros((m + 1, m))
    
    for j in range(m):
        w.append(A.dot(V[j]))
                
        for i in range(j+1):
            h[i][j]=np.inner(w[j],V[i])
            w[j]=w[j]-h[i][j]*V[i]
                        
        h[j+1][j]=la.norm(w[j])
                
        if h[j+1][j]==0:
            return np.array(V).T, h[0:-1]
        
        V.append(w[j]/h[j+1][j])
                
    return np.array(V[0:-1]).T, h[0:-1]

In [117]:
def Linear_Arnoldi(A, v, m):
    #A = LinearOperator((m, m), matvec=mymatvec)
    w=[]
    V = []
    V.append(np.array(v[:]))
    h=np.zeros((m + 1, m))
    
    for j in range(m):
        w.append(A.dot(V[j]))
                
        for i in range(j+1):
            h[i][j]=np.inner(w[j],V[i])
            w[j]=w[j]-h[i][j]*V[i]
                        
        h[j+1][j]=la.norm(w[j])
                
        if h[j+1][j]==0:
            return np.array(V).T, h[0:-1]
        
        V.append(w[j]/h[j+1][j])
    #print('!!!!!!!!!', h[-1][-1])            
    return np.array(V[0:-1]).T, h[0:-1]

In [118]:
def mymatvec(z):
    res = np.zeros((z.size, 1))
    res[0] = (-2-alpha)*z[0]+z[1]
    res[res.size-1] = (-2-alpha)*z[z.size-1]+z[z.size-2]
    for i in range(1,z.size-1):
        res[i] = (-2-alpha)*z[i]+z[i-1]+z[i+1]
    return res

In [119]:
def accurate_vectors(A,v0,m):
    n=np.zeros((m,m))
    n[0,:]=v0
    for i in range (m-1):
        tmp=A.dot(n[i,:])
        n[i+1,:]=tmp/la.norm(tmp)
    return n

In [120]:
def accurate_vectors_lgmres(v0,m):
    dim = len(v0)
    A = LinearOperator((dim, dim), matvec=mymatvec)
    Ainv = LinearOperator((dim,dim), lambda v: lgmres(A, v, tol=10**-10, atol=10**-10)[0])
    n=np.zeros((m,dim))
    n[0,:]=v0
    for i in range (m-1):
        
        tmp=Ainv.dot(n[i,:])
        n[i+1,:]=tmp/la.norm(tmp)
    return n

In [121]:
def approximate_vectors(V,H1,v0,m):
    dim = len(v0)
    n2=np.zeros((m,dim))
    n2[0,:]=v0
    for i in range(m-1):
        tmp=V.dot(H1.dot(V.T.dot(n2[i,:])))
        n2[i+1,:]=tmp/la.norm(tmp)
    return n2

In [122]:
def relative_error(V,H1,n,m):
    print("Без пересчета")
    r=[]
    vect = V[:,0]
    for i in range(0, m-1):
        vect = V.dot(H1.dot(V.T.dot(vect)))
        vect = vect/np.linalg.norm(vect)
        r.append(np.linalg.norm(n[i+1]-vect)/np.linalg.norm(n[i+1]))
    return r

In [126]:
def relative_error_Arnoldi(V, H1, n,m,j, tmp):
    print("Пересчет каждый "+str(j)+" раз")
    r=[]
    #tmp=np.zeros(A.shape[0])
    #tmp[0]=1.
    for i in range(0, m-1):
        if (j-1==0) or i%(j-1)==0:
            V,H=Linear_Arnoldi(A, tmp,m)
            #print(H)
            #print(V.dot(H.dot(V.T)))
            H1=la.inv(H)
        vect = V.dot(H1.dot(V.T.dot(tmp)))#n[i,:])))
        vect2= lgmres(A,tmp)[0]
        #print('%%%%%%%',vect-vect2)
        vect = vect/np.linalg.norm(vect)
        tmp = vect
        #print(np.linalg.norm(n[i+1]))
        r.append(np.linalg.norm(n[i+1]-vect)/np.linalg.norm(n[i+1]))
    return r

In [127]:
alpha=1
m=60
vect_cnt = 30
tmp = np.ones(m)
tmp = tmp/np.linalg.norm(tmp)

In [128]:

A = LinearOperator((m, m), matvec=mymatvec)
v0=tmp
print('@@@@@@@@@@@@@')
V,H=Linear_Arnoldi(A,v0,vect_cnt)
H1=la.inv(H)
v=accurate_vectors_lgmres(V[:,0],vect_cnt)
v2=approximate_vectors(V,H1,V[:,0],vect_cnt)
print(v)
print(v2)
r0=relative_error(V,H1,v,vect_cnt)
print(r0)
r1=relative_error_Arnoldi(V, H1, v,vect_cnt,1,tmp)
print(r1)
r2=relative_error_Arnoldi(V,H1,v,vect_cnt,3,tmp)
print(r2)
r3=relative_error_Arnoldi(V,H1,v,vect_cnt,5,tmp)
print(r3)
r4=relative_error_Arnoldi(V,H1,v,vect_cnt,10,tmp)
print(r4)

@@@@@@@@@@@@@
[[ 0.12909944  0.12909944  0.12909944 ...  0.12909944  0.12909944
   0.12909944]
 [-0.08124329 -0.11227546 -0.12412869 ... -0.12412869 -0.11227546
  -0.08124329]
 [ 0.05947234  0.09622826  0.11563039 ...  0.11563039  0.09622826
   0.05947234]
 ...
 [-0.01647846 -0.03263552 -0.04816932 ... -0.04816932 -0.03263552
  -0.01647846]
 [ 0.01623055  0.03215641  0.04749071 ...  0.04749071  0.03215641
   0.01623055]
 [-0.01599643 -0.03170342 -0.04684785 ... -0.04684785 -0.03170342
  -0.01599643]]
[[ 0.12909944  0.12909944  0.12909944 ...  0.12909944  0.12909944
   0.12909944]
 [-0.08124301 -0.11227507 -0.12412827 ... -0.12412827 -0.11227507
  -0.08124301]
 [ 0.05947188  0.09622752  0.1156295  ...  0.1156295   0.09622752
   0.05947188]
 ...
 [-0.01647894 -0.03263656 -0.04817111 ... -0.04817111 -0.03263656
  -0.01647894]
 [ 0.01623119  0.03215779  0.04749303 ...  0.04749303  0.03215779
   0.01623119]
 [-0.01599725 -0.03170516 -0.04685074 ... -0.04685074 -0.03170516
  -0.01599725]]
Бе

C:\Users\Vlad\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: scipy.sparse.linalg.lgmres called without specifying `atol`. The default value will change in the future. To preserve current behavior, set ``atol=tol``.
  del sys.path[0]


[0.0026193005729849904, 0.0021112668156212586, 0.005010736619543789, 0.009075635987787633, 0.01685732548760979, 0.18683132013438863, 0.07177000553180206, 0.199665677996692, 0.11739551303053768, 0.07013704095358692, 0.04860873438164496, 0.03177188268728181, 0.020863949704566804, 0.02566706118826673, 0.013138183138869492, 0.00874958963872863, 0.07675064645886379, 0.044009067342062294, 0.03160861935675069, 0.028904359906779534, 0.019244946329391794, 0.030439753715512646, 0.022141535975179736, 0.019815240128236104, 0.015240054904993137, 0.02188209199059017, 0.015631783442291956, 0.018167757932707373, 0.017776317541898654]
Пересчет каждый 3 раз
[0.0026193005729849904, 0.003937310963901721, 0.002265742430713355, 0.0015295193361789484, 1.1156406003988615, 1.1649226428847388, 0.553903119648333, 0.19175830862972296, 0.0913517756169216, 0.0530351735491617, 0.03254234488904385, 0.020990549894284286, 0.013936651885313492, 0.010266057923306133, 0.0077753996879041335, 0.0075384270025854435, 0.098319

In [ ]:
dots_x=range(1,40)
plt.plot(dots_x, r0)
plt.plot(dots_x, r1)
plt.plot(dots_x, r2)
plt.plot(dots_x, r3)
plt.plot(dots_x, r4)
plt.show()